# 📘 P1.3.2.6 – Flask Basics
## Topic: Building Simple REST APIs with Flask

## 🎯 Learning Objectives
By the end of this notebook, you will:
- Understand REST API principles
- Build API endpoints using Flask
- Implement CRUD operations (Create, Read, Update, Delete)
- Return JSON responses
- Handle HTTP methods (GET, POST, PUT, DELETE)
- Use proper status codes
- Implement basic error handling
- Work with API documentation

## 🌐 What is a REST API?

### REST Principles
**REST** = Representational State Transfer

REST is a style for building web APIs using standard HTTP methods.

### Key Concepts
- **Resources:** Data entities (users, products, posts, etc.)
- **Endpoints:** URLs that represent resources
- **HTTP Methods:** GET, POST, PUT, DELETE
- **JSON:** Data format for requests/responses
- **Status Codes:** Indicate success or failure

### REST URL Patterns
```
GET    /api/users         → Get all users
GET    /api/users/1       → Get user with ID 1
POST   /api/users         → Create a new user
PUT    /api/users/1       → Update user with ID 1
DELETE /api/users/1       → Delete user with ID 1
```

## 📊 CRUD Operations

### CRUD = Create, Read, Update, Delete

| Operation | HTTP Method | Example |
|-----------|-------------|----------|
| **Create** | POST | POST /api/users – Add new user |
| **Read** | GET | GET /api/users/1 – Get user 1 |
| **Update** | PUT | PUT /api/users/1 – Update user 1 |
| **Delete** | DELETE | DELETE /api/users/1 – Delete user 1 |

### Response Status Codes
- `200 OK` – Request succeeded
- `201 Created` – Resource created successfully
- `204 No Content` – Success, no data returned
- `400 Bad Request` – Client error in request
- `404 Not Found` – Resource not found
- `500 Server Error` – Server error

## 🔨 Building a Simple API

### Basic API Structure
```python
from flask import Flask, jsonify, request

app = Flask(__name__)

# Sample data (in memory)
users = [
    {'id': 1, 'name': 'Alice'},
    {'id': 2, 'name': 'Bob'}
]

# GET all users
@app.route('/api/users', methods=['GET'])
def get_users():
    return jsonify(users), 200

# GET specific user
@app.route('/api/users/<int:user_id>', methods=['GET'])
def get_user(user_id):
    user = next((u for u in users if u['id'] == user_id), None)
    if not user:
        return {'error': 'User not found'}, 404
    return jsonify(user), 200
```

In [ ]:
from flask import Flask, jsonify, request

app = Flask(__name__)

# In-memory database
users = [
    {'id': 1, 'name': 'Alice', 'email': 'alice@example.com'},
    {'id': 2, 'name': 'Bob', 'email': 'bob@example.com'}
]
next_id = 3

# GET all users
@app.route('/api/users', methods=['GET'])
def get_users():
    return jsonify(users), 200

# GET specific user
@app.route('/api/users/<int:user_id>', methods=['GET'])
def get_user(user_id):
    user = next((u for u in users if u['id'] == user_id), None)
    if not user:
        return jsonify({'error': 'User not found'}), 404
    return jsonify(user), 200

# Create new user (POST)
@app.route('/api/users', methods=['POST'])
def create_user():
    global next_id
    data = request.get_json()
    new_user = {'id': next_id, 'name': data.get('name'), 'email': data.get('email')}
    users.append(new_user)
    next_id += 1
    return jsonify(new_user), 201

if __name__ == '__main__':
    app.run(debug=True)

**💡 To see this in action, run:** `app_example1.py` in the folder

Try with curl or Postman:
- `GET http://localhost:5000/api/users` → Get all users
- `GET http://localhost:5000/api/users/1` → Get user 1
- `POST http://localhost:5000/api/users` → Create user (with JSON body)

## ✏️ Update and Delete Operations

### PUT – Update Resource
```python
@app.route('/api/users/<int:user_id>', methods=['PUT'])
def update_user(user_id):
    user = next((u for u in users if u['id'] == user_id), None)
    if not user:
        return {'error': 'User not found'}, 404
    
    data = request.get_json()
    user['name'] = data.get('name', user['name'])
    user['email'] = data.get('email', user['email'])
    return jsonify(user), 200
```

### DELETE – Remove Resource
```python
@app.route('/api/users/<int:user_id>', methods=['DELETE'])
def delete_user(user_id):
    global users
    users = [u for u in users if u['id'] != user_id]
    return '', 204
```

In [ ]:
from flask import Flask, jsonify, request

app = Flask(__name__)

users = [
    {'id': 1, 'name': 'Alice', 'email': 'alice@example.com'},
    {'id': 2, 'name': 'Bob', 'email': 'bob@example.com'}
]

# UPDATE user (PUT)
@app.route('/api/users/<int:user_id>', methods=['PUT'])
def update_user(user_id):
    user = next((u for u in users if u['id'] == user_id), None)
    if not user:
        return jsonify({'error': 'User not found'}), 404
    
    data = request.get_json()
    user['name'] = data.get('name', user['name'])
    user['email'] = data.get('email', user['email'])
    return jsonify(user), 200

# DELETE user
@app.route('/api/users/<int:user_id>', methods=['DELETE'])
def delete_user(user_id):
    global users
    original_len = len(users)
    users = [u for u in users if u['id'] != user_id]
    if len(users) == original_len:
        return jsonify({'error': 'User not found'}), 404
    return '', 204

@app.route('/api/users', methods=['GET'])
def get_users():
    return jsonify(users), 200

if __name__ == '__main__':
    app.run(debug=True)

**💡 To see this in action, run:** `app_example2.py` in the folder

Try with curl or Postman:
- `PUT http://localhost:5000/api/users/1` → Update user (with JSON body)
- `DELETE http://localhost:5000/api/users/1` → Delete user 1
- `GET http://localhost:5000/api/users` → Verify changes

## 🛡️ Error Handling in APIs

### Common Errors
- Missing required fields
- Invalid data type
- Resource not found
- Unauthorized access

### Error Response Format
```python
{
  "error": "User not found",
  "status": 404
}
```

### Best Practices
- Always return appropriate status codes
- Provide clear error messages
- Validate input before processing
- Log errors for debugging

In [ ]:
from flask import Flask, jsonify, request

app = Flask(__name__)

products = [
    {'id': 1, 'name': 'Laptop', 'price': 999.99},
    {'id': 2, 'name': 'Phone', 'price': 499.99}
]
next_id = 3

# Create product with validation
@app.route('/api/products', methods=['POST'])
def create_product():
    global next_id
    data = request.get_json()
    
    # Validation
    if not data:
        return jsonify({'error': 'No JSON data provided'}), 400
    
    if 'name' not in data or not data['name']:
        return jsonify({'error': 'Name is required'}), 400
    
    if 'price' not in data:
        return jsonify({'error': 'Price is required'}), 400
    
    try:
        price = float(data['price'])
        if price < 0:
            return jsonify({'error': 'Price must be positive'}), 400
    except (ValueError, TypeError):
        return jsonify({'error': 'Price must be a number'}), 400
    
    new_product = {'id': next_id, 'name': data['name'], 'price': price}
    products.append(new_product)
    next_id += 1
    return jsonify(new_product), 201

@app.route('/api/products', methods=['GET'])
def get_products():
    return jsonify(products), 200

if __name__ == '__main__':
    app.run(debug=True)

**💡 To see this in action, run:** `app_example3.py` in the folder

Try with invalid data:
- `POST http://localhost:5000/api/products` → Missing fields error
- Body: `{"name": "Camera"}` → Missing price error
- Body: `{"name": "Camera", "price": "invalid"}` → Invalid price error
- Body: `{"name": "Camera", "price": 299.99}` → Success

## 🔐 Authentication Basics

### Why Authentication?
- Protect sensitive data
- Verify user identity
- Control access to resources

### Simple API Key Authentication
```python
@app.route('/api/admin/stats', methods=['GET'])
def admin_stats():
    api_key = request.headers.get('X-API-Key')
    if api_key != 'secret-key-123':
        return {'error': 'Unauthorized'}, 401
    return {'users': 100, 'products': 50}
```

### How It Works
1. Client sends request with API key in header
2. Server validates the API key
3. If valid, process request; if not, return 401 Unauthorized

### Real-World Authentication
- JWT (JSON Web Tokens)
- OAuth 2.0
- Session-based auth

In [ ]:
from flask import Flask, jsonify, request

app = Flask(__name__)

# Valid API keys (in real app, store in database)
VALID_KEYS = ['sk-admin-key-123', 'sk-user-key-456']

def validate_api_key():
    """Check if request has valid API key"""
    api_key = request.headers.get('X-API-Key')
    return api_key in VALID_KEYS

# Public endpoint (no auth required)
@app.route('/api/status', methods=['GET'])
def status():
    return jsonify({'status': 'API is running'}), 200

# Protected endpoint (requires API key)
@app.route('/api/admin/stats', methods=['GET'])
def admin_stats():
    if not validate_api_key():
        return jsonify({'error': 'Unauthorized. Provide X-API-Key header'}), 401
    
    stats = {
        'total_users': 1250,
        'total_products': 500,
        'api_calls_today': 45000
    }
    return jsonify(stats), 200

if __name__ == '__main__':
    app.run(debug=True)

**💡 To see this in action, run:** `app_example4.py` in the folder

Try with curl or Postman:
- `GET http://localhost:5000/api/status` → Works without key
- `GET http://localhost:5000/api/admin/stats` → Returns 401 Unauthorized
- `GET http://localhost:5000/api/admin/stats` with header `X-API-Key: sk-admin-key-123` → Works

## ❌ Common Mistakes

### Mistake 1: Using Wrong HTTP Method
❌ Bad: `@app.route('/api/users')` then POST for create
✅ Good: `@app.route('/api/users', methods=['GET', 'POST'])`

### Mistake 2: Not Returning JSON
❌ Bad: `return {'error': 'Not found'}`
✅ Good: `return jsonify({'error': 'Not found'}), 404`

### Mistake 3: Forgetting Status Codes
❌ Bad: `return jsonify(data)` (always 200)
✅ Good: `return jsonify(data), 201` or appropriate status code

### Mistake 4: No Input Validation
❌ Bad: Assume data is correct
✅ Good: Validate type, length, and required fields

### Mistake 5: Exposing Sensitive Errors
❌ Bad: `return str(error)` (exposes stack trace)
✅ Good: Return generic error message to client

## ✅ Key Takeaways
- REST APIs use standard HTTP methods (GET, POST, PUT, DELETE)
- URLs represent resources: `/api/users`, `/api/products`
- Use proper HTTP status codes: 200, 201, 400, 404, 500
- Always return JSON responses using `jsonify()`
- Validate all input before processing
- Handle errors gracefully with meaningful messages
- Use authentication to protect sensitive endpoints
- Test APIs with tools like Postman or curl
- Document your API for developers
- Follow REST conventions for consistency